<a href="https://colab.research.google.com/github/kgarvita/MISC/blob/main/semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install required packages

##Install Sentence Transformer

In [ ]:
!pip3 install -U sentence-transformers

##Install FAISS

In [ ]:
!pip install faiss-cpu --no-cache

#Import packages and download model

In [ ]:
import pandas as pd
import time
#from tqdm import tqdm
#import seaborn as sns
import numpy as np
#from textblob import TextBlob
#import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
import faiss
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
#model = SentenceTransformer('multi-qa-distilbert-cos-v1')
#model = SentenceTransformer('multi-qa-mpnet-base-dot-v1', cache_folder="content/Model")

#Read CSV file

In [ ]:
data = pd.read_csv('/content/SAP_data.csv',memory_map=True)
data.info()
#data.head(5)

##Clean data

In [ ]:
import gc
df = data[['Incident ID','Incident Description', 'Proposed Fix']]
del data
gc.collect()
df.head(2)

In [ ]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

# Create vectors

In [36]:
encoded_data = model.encode(df.values.tolist())
#encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexFlatL2(encoded_data.shape[1])
index.add(encoded_data)
print(index.is_trained)
#index = faiss.IndexIDMap(faiss.IndexFlatIP(471))
#index.add_with_ids(encoded_data, np.array(range(0, len(df))))
faiss.write_index(index, 'fulldoc.index')
index = faiss.read_index('fulldoc.index')


True


#Search functions

In [30]:
def fetch_info(dataframe_idx):
    info = df.iloc[dataframe_idx]
    print(info)
    meta_dict = {}
    meta_dict['Incident Description'] = info['Incident Description']
    return meta_dict

def search(query, top_k, index, model):
    t=time.time()
    results = []
    query_vector = model.encode([query])
    #print(query_vector.shape)
    top_k = index.search(query_vector, top_k)
    #print("Type of return value of index.search",type(top_k))
    for top in top_k:
      print(top)
    #print('>>>> Results in Total Time: {}'.format(time.time()-t))
    #If search result is not relevant
    max_distance = 1.8
    top_k_distance = top_k[0].tolist()[0]
    for top in top_k_distance:
      print(top)
      if(top > max_distance):
        results.append("No data found, please try to rephrase the question.")
        return results
    top_k_ids = top_k[1].tolist()[0]
    #print(type(top_k_ids))
    for top in top_k_ids:
      print(top)
      print("\n")
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_info(idx) for idx in top_k_ids]
    return results


#Start query loop

In [31]:
while(1):
  print("Please enter your query :")
  query = input()
  if(query == 'exit'):
    print("Thank you for internacting with me.")
    break
  results=search(query,3,index,model)
  #print(type(results))
  #print("\n")
  print("Query: "f"{query}")
  for result in results:
    print('\t',result)

Please enter your query :
Who am I?
[[1.1784923 1.405608  1.407039 ]]
[[2 3 0]]
1.1784923076629639
1.405608057975769
1.4070390462875366
2


3


0


Incident ID                                           INC001
Incident Description    I am not able to login to my system.
Proposed Fix                                     Login page 
Name: 0, dtype: object
Incident ID                           INC003
Incident Description    What is my login ID?
Proposed Fix                   login ID page
Name: 2, dtype: object
Incident ID                                    INC004
Incident Description        How do I create new user?
Proposed Fix            Create user/Subscription page
Name: 3, dtype: object
Query: Who am I?
	 {'Incident Description': 'I am not able to login to my system.'}
	 {'Incident Description': 'What is my login ID?'}
	 {'Incident Description': 'How do I create new user?'}
Please enter your query :
What is the capital?
[[1.7895775 1.7980099 1.8272619]]
[[ 9  2 13]]
1.7895774841308594

Now partition the index into Voronoi cells

when we introduce a new query vector, we first measure its distance between centroids, then restrict our search scope to that centroid’s cell.

In [ ]:
#Create index as a quantizer and feed into partioning
nlist = 10  # how many partitions
dimention = encoded_data.shape[1]
quantizer = faiss.IndexFlatL2(dimention)
index = faiss.IndexIVFFlat(quantizer, dimention, nlist)
print(index.is_trained)

#Train the index
index.train(encoded_data)
print(index.is_trained)  # check if index is now trained

#Add vectors to index
index.add(encoded_data)
print(index.ntotal)  # number of embeddings indexed

faiss.write_index(index, 'fulldoc.index')
index = faiss.read_index('fulldoc.index')


False
True
29


In [ ]:
# Query the index

query="Teams is not loading?"
results=search(query,2,index,model)

print("\n")
for result in results:
    print('\t',result)

(1, 384)
>>>> Results in Total Time: 0.03233599662780762
Ticket ID                                                       INC0001
Ticket Description    My teams say it's offline even though I'm onli...
Name: 0, dtype: object
Ticket ID                                                       INC0002
Ticket Description    Microsoft Teams status stuck on ‘offline’ – ho...
Name: 1, dtype: object


	 {'Ticket Description': "My teams say it's offline even though I'm online, can anyone help me?"}
	 {'Ticket Description': 'Microsoft Teams status stuck on ‘offline’ – how to fix'}


My code starts from here!!

In [ ]:
# Import packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, warnings
warnings.simplefilter('ignore')
import time
from sentence_transformers import SentenceTransformer
import csv
import os
# FAISS library (Facebook AI Similarity Search)
import faiss

# Torch library
import torch

In [ ]:
def write_data(file_name):
  import os.path
  row_num=0
  #df = pd.read_csv(file_name, header=None,encoding='cp1252')
  with open (file_name, 'r',encoding='cp1252') as file:
    csvreader=csv.reader(file)
    for row in csvreader:
      if(row_num == 0):
        header = row
        row_num = row_num + 1
        continue
      folder_name= "/content/data/"f"{row[9]}"
      print(folder_name)
      if(os.path.isdir(folder_name) == False):
        os.mkdir(f"{folder_name}")
      componentfile_name = row[11]
      filepath = f"{folder_name}/"f"{componentfile_name}.csv"
      if(os.path.isfile(filepath) == False):
        with open(filepath, "w", encoding='cp1252')as file:
          writer=csv.writer(file)
          writer.writerow(header)
      with open(filepath, 'a',encoding='cp1252')as file:
        writer=csv.writer(file)
        writer.writerow(row)
write_data("/content/SAPDUMMYDATA.csv")

In [ ]:
#create list from directory
def getlist_dir(folder_path):
  import os
  #dir_path = folder_path
  dirlst = []
# Iterate directory
  for dir_path in os.listdir(folder_path):
    # check if current file_path is a file
      if os.path.isdir(os.path.join(folder_path, dir_path)):
        # add filename to list
          dirlst.append(dir_path)
  return dirlst

In [ ]:
#create list from directory
def getlist_file(folder_path):
  print(folder_path)
  filelst = []
# Iterate directory
  for file_path in os.listdir(folder_path):
    # check if current file_path is a file
      if os.path.isfile(os.path.join(folder_path, file_path)):
        # add filename to list
          filelst.append(file_path)
  return filelst

In [ ]:
# load a pretrained model
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [ ]:
#Load data
dir_path = "/content/data"
filelst=[]
prdlst=[]
dflist=[]
datalist=[]
prdlst = getlist_dir(dir_path)
#print(prdlst)
for prd in prdlst:
  filelst = getlist_file(f"{dir_path}/"f"{prd}/")
  #print(filelst)
  for file in filelst:
    #dfname = "df"f"{file[:-4]}"
    #print(dfname)
    file_path = f"{dir_path}/"f"{prd}/"f"{file}"
    #print(file_path)
    #dfname = pd.read_csv(file_path,encoding='cp1252')
    df = pd.read_csv(file_path,encoding='cp1252')
    #print(df)
    # drop NULLs & reset index
    #df.dropna(inplace=True)
    #df.reset_index(drop=True,inplace=True)
    col = ['Incident date', 'Date of Resolution','Priority', 'SLA in days','SLA Voilated','Status']
    df=df.drop(col,axis = 1)
    #print(df)
  df = pd.DataFrame(df, columns=['Ticket Description', 'Ticket ID'])
  print(df)
  data = df.values.tolist()

   # Create embedding
    #data_embds=model.encode(data)
  data_embds=model.encode(data)
  print(data_embds)
  index = faiss.IndexFlatL2(data_embds.shape[1])
  index.add(data_embds)
  faiss.write_index(index, f"{file[:-4]}"'_index_data')
  index = faiss.read_index(f"{file[:-4]}"'_index_data')


In [ ]:
#Load single file
file="/content/SAPDUMMYDATA.csv"
df = pd.read_csv(file,encoding='cp1252')
# drop NULLs & reset index
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
print(df.head)


In [ ]:
# load a pretrained model
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [ ]:
# pre-process text data
data = df['Ticket Description'].tolist()
# embed patient findings
data_embds=model.encode(data)
print(data_embds)

In [ ]:
# Create an index using FAISS
index = faiss.IndexFlatL2(data_embds.shape[1])
index.add(data_embds)
faiss.write_index(index, 'index_data')
index = faiss.read_index('index_data')

In [ ]:
# Perform semantic search
def search(query):
    query_vector = model.encode([query])
    k = 3
    #top_k = index.search(query_vector, k)
    #index = faiss.read_index('office_index_data')
    top_k = index.search(query_vector, k)
    print(type(top_k))
    return [data[_id] for _id in top_k[1].tolist()[0]]
   # results
qry = 'Which browsers are supported? - SAP for Me '
results = search(qry)
print("\n\n======================\n\n")
print("Query:", qry)
print("\nTop 5 most similar sentences :\n")
for r in enumerate(results, start=1):
    print(f"{r[0]}). {r[1]}")

<class 'tuple'>




Query: Which browsers are supported? - SAP for Me 

Top 5 most similar sentences :

1). ['Outlook shows  "Processing," stopped working, not responding', 'INC0004']
2). ['Outlook shows  "Processing," stopped working, not responding', 'INC0004']
3). ['Outlook shows  "Processing," stopped working, not responding', 'INC0004']
